In [1]:
# Process outputs to get resource values
# Requires the non-instrumented binary in most cases

# Supports:
#   * Output: Total size of program stdout
#   * Trace: Size of program traces (could support count of individual funcs)
#   * Tcpdump: Different measures based on live tcp capture

In [2]:
import os
import random
from tqdm import tqdm
import pickle

In [3]:
def make_program_string(args, file):
    resource = ""
    program = ""
    if args.resource_type == 'trace':
        resource = f"{args.dynamo_rio}drltrace -logdir logs -- "
    elif args.resource_type == 'output_lines':
        resource = "" # Only normal output
            
    if args.file_input:
        program = f"{args.program_string} {args.fuzzing_output}{file}"
    else:
        assert False, "Not Implemented"
    
    return resource + program

def make_tail(args):
    return None # Do not need

In [4]:
Args = type('Args', (object,), {})
args = Args()

args.set_name = "out_100k"
args.fuzzing_output = "./programs/coreutils/out_100k/default/queue_all/"
args.file_input = True # If False then stdin to the program
args.program_string = "./programs/coreutils/clean/tar-1.35/src/tar --list -f "
args.resource_type = "output_size"  # choices=['output_size', 'trace']
args.file_limit = 1000 # Maximum number of files to process (default=0, all files)
args.shuffle = True # Randomize the file processing order
args.dynamo_rio = "./dynamorio/"

In [5]:
files = os.listdir(args.fuzzing_output)
assert len(files) > 0, "No fuzzing output files found"
if args.shuffle: random.shuffle(files)
print(f"Loaded {len(files)} fuzzing output files")
if args.file_limit > 0:
    files = files[:args.file_limit]
    print(f"Taking {len(files)} files")

Loaded 105835 fuzzing output files
Taking 1000 files


In [6]:
print("Testing program functionality")
output = !{args.program_string} {args.fuzzing_output}{files[0]}
assert not "No such file" in output[0], "File not found"
output

Testing program functionality


['./programs/coreutils/clean/tar-1.35/src/tar: This does not look like a tar archive',
 './programs/coreutils/clean/tar-1.35/src/tar: Skipping to next header',
 './programs/coreutils/clean/tar-1.35/src/tar: A lone zero block at 16',
 './programs/coreutils/clean/tar-1.35/src/tar: Exiting with failure status due to previous errors']

In [7]:
#!rm logs/*log

In [8]:
resources = []
for file in tqdm(files):
    vs = !{make_program_string(args, file)}
    sz = 0
    for v in vs: sz += len(v)
    resources.append(sz)
    
    #sz = !wc -c logs/*log
    #sz = sz[0].split(" ")
    #sz = int(sz[0])
    #sizes.append(sz)
    #!rm logs/*log

100%|██████████| 1000/1000 [00:02<00:00, 399.17it/s]


In [9]:
max(resources)

643

In [10]:
def saveData(obj, name):
    with open('./data/' +  name + '.pkl', 'wb') as f:
        pickle.dump(obj, f)
        

In [11]:
saveData({"labels":resources}, args.set_name)